# CiRA Demonstration

This notebook demonstrates the usage of the CiRA pipeline. Be sure to fulfill all prerequisites as specified in the [readme](README.md), especially the availability of the two pretrained models, both of which are [available for download on Zenodo](https://doi.org/10.5281/zenodo.7186287). If you run the code from the provided container, the models are already included.

## Locate the Models

The location of the two pre-trained models must be available to the CiRA converter. You can hardcode the location of the pre-trained models to the two variables `model_classification` and `model_labeling`, or use the subsequent snippet, in which the location of the models is determined. If you run the code locally, 

1. create a `.env` file and 
2. define the variables `MODEL_CLASSIFICATION` and `MODEL_LABELING` with the respective URLs. 

If you run the code from the container, then the suffix `_DEV` in the environment variables will distinguish the location of the models locally from those in the container.

In [ ]:
import os, dotenv
dotenv.load_dotenv()

model_env_suffix = '_DEV' if ('DEV_CONTAINER' in os.environ) else ''

model_classification = os.environ[f'MODEL_CLASSIFICATION{model_env_suffix}']
model_labeling = os.environ[f'MODEL_LABELING{model_env_suffix}']

## Create the Converter

Instantiate the CiRA converter which is realized in [src.cira.CiRAConverter](./src/cira.py). The instantiation of the converter will output some warning messages, which can be safely ignored.

In [ ]:
from src.cira import CiRAConverter
cira = CiRAConverter(classifier_causal_model_path=model_classification, converter_s2l_model_path=model_labeling)

## Apply the converter

Finally, apply the converter to utilize the CiRA pipeline. This usually includes four steps:

1. Define a single natural language sentence. CiRA is currently only trained on grammatically correct, full sentences.
2. Classify the sentence to determine, whether the sentence is causal or nor.
3. If the sentence is causal, process it to generate labels, convert it into a cause-effect-graph, and finally derive a minimal set of test cases covering the requirement.

Non-causal sentences cannot be processed by the pipeline.

In [ ]:
sentence: str = "If the red button is pressed and background operations are not running then the system shuts down."

In [ ]:
causal, confidence = cira.classify(sentence)
print(f'CiRA classified the sentence "{sentence}" to be {"*" if causal else "*non"}causal* with a confidence of {confidence:.2%}.')

In [ ]:
if causal:
    labels, graph, suite = cira.process(sentence)

    print(f'Labels associated to the sentence: {labels}\n')
    print(f'Converted cause-effect-graph: {graph}\n')
    print(f'Minimal test suite: \n{suite}')